In [9]:
import requests
import json
import pandas as pd
from pprint import pprint

states = ["Alabama", "Alaska", "Arizona","Arkansas","California","Colorado","Connecticut","Delaware","District of Columbia","Florida","Georgia",
          "Hawaii","Idaho","Illinois","Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts",
          "Michigan","Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada","New Hampshire","New Jersey",
          "New Mexico","New York","North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania","Rhode Island",
          "South Carolina","South Dakota","Tennessee","Texas","Utah","Vermont","Virginia","Washington","West Virginia","Wisconsin",
          "Wyoming"
]

In [10]:
df = pd.DataFrame(data={"States": states})
df["Median Income"] = ""
df["Home Value"] = ""
df["Bachelors Degree"] = ""
df["Population"] = ""
df["Divorce"] = ""


df

,States,Median Income,Home Value,Bachelors Degree,Population,Divorce
0,Alabama,,,,,
1,Alaska,,,,,
2,Arizona,,,,,
3,Arkansas,,,,,
4,California,,,,,
5,Colorado,,,,,
6,Connecticut,,,,,
7,Delaware,,,,,
8,District of Columbia,,,,,
9,Florida,,,,,
